In [119]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import re
import requests
import seaborn as sns
import time
import descartes
import geopandas as gpd

from arcgis.geocoding import geocode
from arcgis.gis import GIS
from dotenv import load_dotenv
from functools import reduce
from shapely.geometry import Point, Polygon
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.svm import SVC, SVR
from textwrap import wrap
from xgboost import XGBClassifier, XGBRegressor, plot_importance
from yellowbrick.classifier import ROCAUC, ConfusionMatrix

plt.style.use("ggplot")
sns.set_context("talk")

In [2]:
load_dotenv("")
censuskey = os.getenv("censuskey")

In [207]:
var_description = pd.DataFrame(pd.DataFrame(
    requests.get("https://api.census.gov/data/2016/acs/acs5/profile/variables.json")
                                            .json()["variables"])
                               .T.iloc[:, 3]
                               .str.title()
                               .str.split("!!", expand=True)
                              ).sort_index()

In [208]:
var_description.columns=["unit", 
                         "subject",
                         "denominator",
                         "criteria_1",
                         "criteria_2",
                         "criteria_3",
                         "criteria_4"
                        ]

In [213]:
var_description

,unit,subject,denominator,criteria_1,criteria_2,criteria_3,criteria_4
AIANHH,American Indian Area/Alaska Native Area/Hawaii...,None,None,None,None,None,None
AIHHTLI,American Indian Area (Off-Reservation Trust La...,None,None,None,None,None,None
AITSCE,American Indian Tribal Subdivision (Census),None,None,None,None,None,None
ANRC,Alaska Native Regional Corporation,None,None,None,None,None,None
CBSA,Metropolitan Statistical Area/Micropolitan Sta...,None,None,None,None,None,None
CD115,Congressional District (115Th),None,None,None,None,None,None
CNECTA,Combined New England City And Town Area,None,None,None,None,None,None
CONCIT,Consolidated City,None,None,None,None,None,None
COUNTY,County (Fips),None,None,None,None,None,None
COUSUB,County Subdivision (Fips),None,None,None,None,None,None


In [210]:
var_description.to_csv("../data/var_description.csv")

In [211]:
variables_selected = var_description.loc[((~var_description.index.str.contains(r'DP')|
                                           (var_description.index.str.contains(r'DP')&
                                            var_description.index.str.contains(r'[0-9]PE')))&
                                          ~var_description.index.str.contains(r'PR_'))
                                        ].sort_index()

In [109]:
len(variables_selected)

554

In [193]:
varlist_dict = {}
var_dict = {}
for year in range(2010, 2018):
    varlist_dict[year] = []
    k = 0
    for var in variables_selected.index:
        success = False
        l = 0
        while not success:
            l += 1
            resp = requests.get(
                "https://api.census.gov/data/{}/acs/acs5/profile?get={}&for=county:*&in=state:*&key={}"
                .format(year, var, censuskey))
            time.sleep(1)
            if resp.status_code ==200:
                varlist_dict[year].append(pd.DataFrame(resp.json()[1:], columns=resp.json()[0]))
                success = True
            elif l > 10:
                success = True
        k += 1
        
        if k%int(len(variables_selected)/10)==0:
            print("---progress: {:0.2f}% completed for {}---".format(
                k*10/int(len(variables_selected)/10), 
                year
            ))
    var_dict[year] = reduce(lambda left, right: pd.merge(left, 
                                                         right, 
                                                         how='outer'
                                                        ), varlist_dict[year])
    print(year, var_dict[year].shape)

---progress: 10.00% completed for 2010---
---progress: 20.00% completed for 2010---
---progress: 30.00% completed for 2010---
---progress: 40.00% completed for 2010---
---progress: 50.00% completed for 2010---
---progress: 60.00% completed for 2010---
---progress: 70.00% completed for 2010---
---progress: 80.00% completed for 2010---
---progress: 90.00% completed for 2010---
---progress: 100.00% completed for 2010---
2010 (3221, 544)
---progress: 10.00% completed for 2011---
---progress: 20.00% completed for 2011---
---progress: 30.00% completed for 2011---
---progress: 40.00% completed for 2011---
---progress: 50.00% completed for 2011---
---progress: 60.00% completed for 2011---
---progress: 70.00% completed for 2011---
---progress: 80.00% completed for 2011---
---progress: 90.00% completed for 2011---
---progress: 100.00% completed for 2011---
2011 (3221, 545)
---progress: 10.00% completed for 2012---
---progress: 20.00% completed for 2012---
---progress: 30.00% completed for 2012--

In [258]:
for year in range(2010, 2018):
    var_dict[year] = var_dict[year].apply(pd.to_numeric, errors='coerce')
    var_dict[year].to_csv("../data/census_{}.csv".format(year))

In [257]:
var_dict[2016].apply(pd.to_numeric, errors='coerce').describe()

,AIANHH,state,county,AIHHTLI,AITSCE,ANRC,CBSA,CD115,CNECTA,CONCIT,COUNTY,COUSUB,CSA,DIVISION,DP02_0001PE,DP02_0002PE,DP02_0003PE,DP02_0004PE,DP02_0005PE,DP02_0006PE,DP02_0007PE,DP02_0008PE,DP02_0009PE,DP02_0010PE,DP02_0011PE,DP02_0012PE,DP02_0013PE,DP02_0014PE,DP02_0015PE,DP02_0016PE,DP02_0017PE,DP02_0018PE,DP02_0019PE,DP02_0020PE,DP02_0021PE,DP02_0022PE,DP02_0023PE,DP02_0024PE,DP02_0025PE,DP02_0026PE,DP02_0027PE,DP02_0028PE,DP02_0029PE,DP02_0030PE,DP02_0031PE,DP02_0032PE,DP02_0033PE,DP02_0034PE,DP02_0035PE,DP02_0036PE,DP02_0037PE,DP02_0038PE,DP02_0039PE,DP02_0040PE,DP02_0041PE,DP02_0042PE,DP02_0043PE,DP02_0044PE,DP02_0045PE,DP02_0046PE,DP02_0047PE,DP02_0048PE,DP02_0049PE,DP02_0050PE,DP02_0051PE,DP02_0052PE,DP02_0053PE,DP02_0054PE,DP02_0055PE,DP02_0056PE,DP02_0057PE,DP02_0058PE,DP02_0059PE,DP02_0060PE,DP02_0061PE,DP02_0062PE,DP02_0063PE,DP02_0064PE,DP02_0065PE,DP02_0066PE,DP02_0067PE,DP02_0068PE,DP02_0069PE,DP02_0070PE,DP02_0071PE,DP02_0072PE,DP02_0073PE,DP02_0074PE,DP02_0075PE,DP02_0076PE,DP02_0077PE,DP02_0078PE,DP02_0079PE,DP02_0080PE,DP02_0081PE,DP02_0082PE,DP02_0083PE,DP02_0084PE,DP02_0085PE,DP02_0086PE,DP02_0087PE,DP02_0088PE,DP02_0089PE,DP02_0090PE,DP02_0091PE,DP02_0092PE,DP02_0093PE,DP02_0094PE,DP02_0095PE,DP02_0096PE,DP02_0097PE,DP02_0098PE,DP02_0099PE,DP02_0100PE,DP02_0101PE,DP02_0102PE,DP02_0103PE,DP02_0104PE,DP02_0105PE,DP02_0106PE,DP02_0107PE,DP02_0108PE,DP02_0109PE,DP02_0110PE,DP02_0111PE,DP02_0112PE,DP02_0113PE,DP02_0114PE,DP02_0115PE,DP02_0116PE,DP02_0117PE,DP02_0118PE,DP02_0119PE,DP02_0120PE,DP02_0121PE,DP02_0122PE,DP02_0123PE,DP02_0124PE,DP02_0125PE,DP02_0126PE,DP02_0127PE,DP02_0128PE,DP02_0129PE,DP02_0130PE,DP02_0131PE,DP02_0132PE,DP02_0133PE,DP02_0134PE,DP02_0135PE,DP02_0136PE,DP02_0137PE,DP02_0138PE,DP02_0139PE,DP02_0140PE,DP02_0141PE,DP02_0142PE,DP02_0143PE,DP02_0144PE,DP02_0145PE,DP02_0146PE,DP02_0147PE,DP02_0148PE,DP02_0149PE,DP02_0150PE,DP02_0151PE,DP02_0152PE,DP03_0001PE,DP03_0002PE,DP03_0003PE,DP03_0004PE,DP03_0005PE,DP03_0006PE,DP03_0007PE,DP03_0008PE,DP03_0009PE,DP03_0010PE,DP03_0011PE,DP03_0012PE,DP03_0013PE,DP03_0014PE,DP03_0015PE,DP03_0016PE,DP03_0017PE,DP03_0018PE,DP03_0019PE,DP03_0020PE,DP03_0021PE,DP03_0022PE,DP03_0023PE,DP03_0024PE,DP03_0025PE,DP03_0026PE,DP03_0027PE,DP03_0028PE,DP03_0029PE,DP03_0030PE,DP03_0031PE,DP03_0032PE,DP03_0033PE,DP03_0034PE,DP03_0035PE,DP03_0036PE,DP03_0037PE,DP03_0038PE,DP03_0039PE,DP03_0040PE,DP03_0041PE,DP03_0042PE,DP03_0043PE,DP03_0044PE,DP03_0045PE,DP03_0046PE,DP03_0047PE,DP03_0048PE,DP03_0049PE,DP03_0050PE,DP03_0051PE,DP03_0052PE,DP03_0053PE,DP03_0054PE,DP03_0055PE,DP03_0056PE,DP03_0057PE,DP03_0058PE,DP03_0059PE,DP03_0060PE,DP03_0061PE,DP03_0062PE,DP03_0063PE,DP03_0064PE,DP03_0065PE,DP03_0066PE,DP03_0067PE,DP03_0068PE,DP03_0069PE,DP03_0070PE,DP03_0071PE,DP03_0072PE,DP03_0073PE,DP03_0074PE,DP03_0075PE,DP03_0076PE,DP03_0077PE,DP03_0078PE,DP03_0079PE,DP03_0080PE,DP03_0081PE,DP03_0082PE,DP03_0083PE,DP03_0084PE,DP03_0085PE,DP03_0086PE,DP03_0087PE,DP03_0088PE,DP03_0089PE,DP03_0090PE,DP03_0091PE,DP03_0092PE,DP03_0093PE,DP03_0094PE,DP03_0095PE,DP03_0096PE,DP03_0097PE,DP03_0098PE,DP03_0099PE,DP03_0100PE,DP03_0101PE,DP03_0102PE,DP03_0103PE,DP03_0104PE,DP03_0105PE,DP03_0106PE,DP03_0107PE,DP03_0108PE,DP03_0109PE,DP03_0110PE,DP03_0111PE,DP03_0112PE,DP03_0113PE,DP03_0114PE,DP03_0115PE,DP03_0116PE,DP03_0117PE,DP03_0118PE,DP03_0119PE,DP03_0120PE,DP03_0121PE,DP03_0122PE,DP03_0123PE,DP03_0124PE,DP03_0125PE,DP03_0126PE,DP03_0127PE,DP03_0128PE,DP03_0129PE,DP03_0130PE,DP03_0131PE,DP03_0132PE,DP03_0133PE,DP03_0134PE,DP03_0135PE,DP03_0136PE,DP03_0137PE,DP04_0001PE,DP04_0002PE,DP04_0003PE,DP04_0004PE,DP04_0005PE,DP04_0006PE,DP04_0007PE,DP04_0008PE,DP04_0009PE,DP04_0010PE,DP04_0011PE,DP04_0012PE,DP04_0013PE,DP04_0014PE,DP04_0015PE,DP04_0016PE,DP04_0017PE,DP04_0018PE,DP04_0019PE,DP04_0020PE,DP04_0021PE,DP04_0022PE,DP04_0023PE,DP04_0024PE,DP04_0025PE,DP04_0026PE,DP04_0027PE,DP04_0028PE,DP04_0029PE,DP04_0030PE,DP04_0031PE,DP04_0032PE,DP04_0033PE,DP04_0034PE,DP04_0035PE,DP04_0036PE,DP04_00

In [243]:
numeric_2016 = var_dict[2016].loc[:, pd.to_numeric(var_dict[2016].mean(), errors="coerce").dropna().index]

In [260]:
pd.concat([var_dict[2016].loc[:,
                             (var_dict[2016].max()>100)|
                             (var_dict[2016].min()<0)
                            ].max(),
            var_dict[2016].loc[:,
                             (var_dict[2016].max()>100)|
                             (var_dict[2016].min()<0)
                            ].min(),
            var_dict[2016].loc[:,
                             (var_dict[2016].max()>100)|
                             (var_dict[2016].min()<0)
                            ].mean()],
         axis=1)

,0,1,2
county,840.0,1.0,1.029220e+02
CBSA,99999.0,10100.0,5.857850e+04
COUNTY,840.0,1.0,1.029220e+02
CSA,999.0,104.0,7.458034e+02
DP02_0001PE,3281845.0,37.0,3.746538e+04
DP02_0015PE,-888888888.0,-888888888.0,-8.888889e+08
DP02_0016PE,-888888888.0,-888888888.0,-8.888889e+08
DP02_0017PE,9881693.0,76.0,9.881698e+04
DP02_0024PE,3987136.0,37.0,3.997037e+04
DP02_0030PE,4177105.0,33.0,4.198963e+04


In [261]:
var_dict[2016].loc[:,(var_dict[2016].max()>100)|(var_dict[2016].min()<0)].describe()

,county,CBSA,COUNTY,CSA,DP02_0001PE,DP02_0015PE,DP02_0016PE,DP02_0017PE,DP02_0024PE,DP02_0030PE,DP02_0036PE,DP02_0037PE,DP02_0038PE,DP02_0039PE,DP02_0040PE,DP02_0041PE,DP02_0042PE,DP02_0043PE,DP02_0044PE,DP02_0045PE,DP02_0046PE,DP02_0047PE,DP02_0048PE,DP02_0049PE,DP02_0050PE,DP02_0051PE,DP02_0052PE,DP02_0058PE,DP02_0068PE,DP02_0070PE,DP02_0072PE,DP02_0074PE,DP02_0076PE,DP02_0078PE,DP02_0086PE,DP02_0093PE,DP02_0094PE,DP02_0095PE,DP02_0096PE,DP02_0097PE,DP02_0098PE,DP02_0099PE,DP02_0100PE,DP02_0101PE,DP02_0102PE,DP02_0103PE,DP02_0104PE,DP02_0105PE,DP02_0106PE,DP02_0107PE,DP02_0108PE,DP02_0109PE,DP02_0110PE,DP02_0122PE,DP02_0150PE,DP02_0151PE,DP02_0152PE,DP03_0001PE,DP03_0008PE,DP03_0010PE,DP03_0014PE,DP03_0015PE,DP03_0016PE,DP03_0017PE,DP03_0018PE,DP03_0025PE,DP03_0026PE,DP03_0032PE,DP03_0046PE,DP03_0051PE,DP03_0062PE,DP03_0063PE,DP03_0065PE,DP03_0067PE,DP03_0069PE,DP03_0071PE,DP03_0073PE,DP03_0075PE,DP03_0086PE,DP03_0087PE,DP03_0088PE,DP03_0089PE,DP03_0090PE,DP03_0091PE,DP03_0092PE,DP03_0093PE,DP03_0094PE,DP03_0095PE,DP03_0100PE,DP03_0102PE,DP03_0103PE,DP03_0104PE,DP03_0109PE,DP03_0110PE,DP03_0111PE,DP03_0112PE,DP03_0113PE,DP03_0114PE,DP03_0120PE,DP03_0121PE,DP03_0123PE,DP03_0124PE,DP03_0125PE,DP03_0126PE,DP03_0127PE,DP03_0129PE,DP03_0130PE,DP03_0131PE,DP03_0132PE,DP04_0001PE,DP04_0004PE,DP04_0005PE,DP04_0006PE,DP04_0016PE,DP04_0027PE,DP04_0037PE,DP04_0038PE,DP04_0045PE,DP04_0048PE,DP04_0049PE,DP04_0050PE,DP04_0057PE,DP04_0062PE,DP04_0072PE,DP04_0073PE,DP04_0075PE,DP04_0076PE,DP04_0080PE,DP04_0089PE,DP04_0090PE,DP04_0093PE,DP04_0094PE,DP04_0095PE,DP04_0096PE,DP04_0097PE,DP04_0098PE,DP04_0099PE,DP04_0100PE,DP04_0101PE,DP04_0102PE,DP04_0109PE,DP04_0110PE,DP04_0111PE,DP04_0112PE,DP04_0113PE,DP04_0114PE,DP04_0115PE,DP04_0116PE,DP04_0117PE,DP04_0125PE,DP04_0126PE,DP04_0134PE,DP04_0135PE,DP04_0136PE,DP04_0143PE,DP05_0001PE,DP05_0017PE,DP05_0022PE,DP05_0025PE,DP05_0028PE,DP05_0058PE,DP05_0065PE,DP05_0081PE,DP05_0082PE,METDIV
count,3220.000000,3220.000000,3220.000000,3220.000000,3.142000e+03,3142.0,3142.0,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3142.0,3142.0,3142.0,3142.0,3142.0,3142.000000,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3142.000000,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3142.000000,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3142.0,3142.0,3142.0,3.220000e+03,3.220000e+03,3.220000e+03,3220.000000,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3220.0,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3220.0,3220.0,3220.0,3220.0,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3220.0,3220.0,3220.0,3.220000e+03,3220.0,3220.0,3220.0,3220.0,3220.0,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3220.000000,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3220.0,3220.0,3.220000e+03,3.220000e+03,3.220000e+03,3220.0,3.220000e+03,3.220000e+03,3220.0,3220.0,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3220.0,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3220.0,3220.000000,3220.0,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3220.0,3220.000000,3220.0,3.220000e+03,3220.0,3220.0,3.220000e+03,3220.0,3.220000e+03,3220.0,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3220.0,3.220000e+03,3220.00000
mean,102.922050,58578.496584,102.922050,745.803416,3.746538e+04,-888888888.0,-888888888.0,9.881698e+04,3.997037e+04,4.198963e+04,-7.071404e

In [262]:
var_dict[2016].loc[:,var_dict[2016].min()==-666666666.0].describe()

,DP02_0044PE,DP02_0045PE,DP02_0046PE,DP02_0047PE,DP02_0048PE,DP02_0050PE,DP02_0051PE,DP02_0094PE,DP02_0095PE,DP02_0098PE,DP02_0099PE,DP02_0101PE,DP02_0102PE,DP02_0104PE,DP02_0105PE,DP02_0106PE,DP02_0107PE,DP02_0108PE,DP02_0109PE,DP03_0015PE,DP03_0017PE,DP03_0110PE,DP03_0111PE,DP03_0112PE,DP03_0113PE,DP03_0120PE,DP03_0121PE,DP03_0123PE,DP03_0124PE,DP03_0125PE,DP03_0126PE,DP03_0127PE,DP03_0129PE,DP03_0130PE,DP03_0131PE,DP03_0132PE,DP04_0094PE,DP04_0095PE,DP04_0096PE,DP04_0097PE,DP04_0098PE,DP04_0099PE,DP04_0100PE,DP04_0111PE,DP04_0112PE,DP04_0113PE,DP04_0114PE,DP04_0115PE
count,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.142000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03,3.220000e+03
mean,-3.819175e+06,-3.819213e+06,-3.819213e+06,-3.819216e+06,-3.819204e+06,-1.082107e+07,-1.082106e+07,-2.758285e+06,-2.758271e+06,-1.591342e+07,-1.591334e+07,-2.758314e+06,-2.758243e+06,-2.758310e+06,-2.758304e+06,-2.758324e+06,-2.758327e+06,-2.758280e+06,-2.758323e+06,-4.140139e+05,-2.069689e+05,-2.070335e+06,-2.070359e+06,-2.070367e+06,-2.070352e+06,-2.070187e+05,-1.449252e+06,-4.140686e+05,-2.277423e+06,-2.070047e+05,-4.140337e+05,-1.739125e+07,-2.070156e+05,-2.070160e+05,-4.140515e+05,-2.070174e+05,-2.070352e+05,-2.070015e+05,-2.070079e+05,-2.070249e+05,-2.070332e+05,-2.070365e+05,-2.070362e+05,-2.069929e+05,-2.070243e+05,-2.070292e+05,-2.070325e+05,-2.070178e+05
std,5.032265e+07,5.032264e+07,5.032264e+07,5.032264e+07,5.032265e+07,8.425706e+07,8.425706e+07,4.280024e+07,4.280024e+07,1.017792e+08,1.017793e+08,4.280023e+07,4.280024e+07,4.280023e+07,4.280024e+07,4.280023e+07,4.280023e+07,4.280024e+07,4.280023e+07,1.661225e+07,1.174846e+07,3.709991e+07,3.709991e+07,3.709991e+07,3.709991e+07,1.174846e+07,3.105451e+07,1.661225e+07,3.890465e+07,1.174846e+07,1.661225e+07,1.062791e+08,1.174846e+07,1.174846e+07,1.661225e+07,1.174846e+07,1.174846e+07,1.174846e+07,1.174846e+07,1.174846e+07,1.174846e+07,1.174846e+07,1.174846e+07,1.174846e+07,1.174846e+07,1.174846e+07,1.174846e+07,1.174846e+07
min,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08,-6.666667e+08
25%,3.790000e+01,4.700000e+00,5.300000e+00,3.100000e+00,1.190000e+01,5.460000e+01,6.480000e+01,3.010000e+01,4.540000e+01,0.000000e+00,8.750000e+01,5.700000e+00,8.100000e+01,6.400000e+00,9.325000e+00,0.000000e+00,0.000000e+00,2.450000e+01,2.000000e-01,5.850000e+01,6.550000e+01,4.770000e+01,2.400000e+01,1.720000e+01,2.980000e+01,1.350000e+01,1.330000e+01,5.200000e+00,2.000000e+00,2.697500e+01,3.550000e+01,3.670000e+01,1.560000e+01,1.510000e+01,1.750000e+01,1.390000e+01,1.700000e+00,2.700000e+01,2.710000e+01,9.300000e+00,2.700000e+00,7.000000e-01,6.000000e-01,4.060000e+01,1.330000e+01,8.500000e+00,5.500000e+00,1.680000e+01
50%,4.765000e+01,8.500000e+00,9.000000e+00,6.300000e+00,1.820000e+01,6.015000e+01,7.370000e+01,4.210000e+01,5.790000e+01,4.000000e+00,9.520000e+01,1.150000e+01,8.840000e+01,1.420000e+0